## effect size sharing for multi-gene qtls

say we have a qtl that impacts multiple genes. How does the fractional effect of that qtl on each gene vary? w.r.t for instance distance from qtl to tss? maybe tie in ABC connections here? Maybe build a model that predicts how qtl impact on each gene will be split based on gene baseline expression, distance, ect

In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import os
import ast
import re

import sys
sys.path.append('/home/klawren/oak/pcqtls/workflow/scripts')
from notebook_helper_functions import *
from annotate_clusters import *

import upsetplot as up
from tqdm.auto import tqdm  # for notebooks

# Create new `pandas` methods which use `tqdm` progress
# (can use tqdm_gui, optional kwargs, etc.)
tqdm.pandas()

In [2]:
# get outputs from a config file
prefix = '/home/klawren/oak/pcqtls'
import yaml
config_path= f'{prefix}/config/proteincoding_main.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# load in the tissue ids 
tissue_df = load_tissue_df(config)
tissue_ids = list(tissue_df['Tissue'])

# select just 1 tissue id to do for now 
my_tissue_id = 'Cells_Cultured_fibroblasts' # because only one cell type
#tissue_ids = [tissue_id]

### load in data
effect sizes for qtls
overlap dfs
gencode for tss starts

In [3]:
e_nominal = load_e_nominal(config, my_tissue_id, chr_id=22)
pc_nominal = load_pc_nominal(config, my_tissue_id, chr_id=22)
overlap = load_overlap(config, my_tissue_id)
pc_susie = load_pc_susie(config, my_tissue_id)

# set e_nominal to be index based on variant-cluster pairs
e_nominal_cid = e_nominal.set_index('var_cluster')

In [4]:
# look at just one chr to start
pc_susie['chr'] = pc_susie['variant_id'].str.split('chr').str[1].str.split('_').str[0]
pc_susie = pc_susie[pc_susie['chr'] == '22']

take the subset of varaints in pcqtl determined credible sets, and calculate the varience-explained on a per gene basis for each gene in the cluster and for the cluster-pc overall

variance for each egene-cluster-credibleset(pc and cs num)

In [38]:
def add_variance(nominal_df):
    nominal_df['variance'] = nominal_df['slope'].apply(np.square) * 100

def get_var_explained(var_list, pip_list, nominal_df, cluster_id):
    # make sure all varaints are in the nominal df
    try:
        nominal_df['variance']
    except KeyError:
        add_variance(nominal_df)
    sub_nominal = nominal_df[(nominal_df['cluster_id'] == cluster_id)&nominal_df['variant_id'].isin(var_list)]
    # add pips to the df    
    sub_nominal = sub_nominal.merge(pd.Series(pip_list, index=pd.Series(var_list, name='variant_id'), name='pip'), on='variant_id')
    sub_nominal['variance_weighted'] = sub_nominal['variance'] * sub_nominal['pip']
    # sum over each phenotype to get the total weighted varience
    return sub_nominal.groupby('phenotype_id').agg({'variance_weighted':'sum'}).reset_index()

In [34]:
cs_grouped_pc_susie = pc_susie.groupby('cs_id').agg({'pip':list, 
                               'variant_id': list,
                               'cs_num':'first',
                               'pc_num':'first',
                               'lead_variant_id':'first',
                               'num_vars':'first',
                               'cluster_id':'first'})

In [61]:
for idx, row in tqdm(cs_grouped_pc_susie.iterrows(), total=len(cs_grouped_pc_susie)):
    pc_var_weighted = get_var_explained(row['variant_id'], row['pip'], pc_nominal, row['cluster_id'])
    cs_grouped_pc_susie.loc[idx, 'matching_pc_var'] = pc_var_weighted[pc_var_weighted['phenotype_id'].str.split('_pc').str[-1] == str(row['pc_num'])].iloc[0]['variance_weighted']

    e_var_weighted = get_var_explained(row['variant_id'], row['pip'], e_nominal, row['cluster_id'])
    cs_grouped_pc_susie.loc[idx, 'e_vars'] = str(e_var_weighted['variance_weighted'].to_list())
    cs_grouped_pc_susie.loc[idx, 'e_var_phenotypes'] = str(e_var_weighted['phenotype_id'].to_list())


  0%|          | 0/35 [00:00<?, ?it/s]

In [63]:
full_gencode, gid_gencode = load_gencode()

In [11]:
# load in the gene information (start and strand are what I need)
full_gencode=pd.read_csv('/home/klawren/oak/pcqtls/data/references/processed_gencode.v26.GRCh38.genes.gtf', sep='\t', skiprows=range(6), 
            header=None, names=['chr', 'dataset', 'type', 'start','end', '.', 'strand', 'na', 'info'])

full_gencode = full_gencode[full_gencode['type']=='transcript']
full_gencode['transcript_id'] = full_gencode['info'].str.split(';').str[1].str.split('\"').str[-2]

# add in the start and end info
full_gencode['tss_start'] = np.where(full_gencode['strand'] == '+', full_gencode['start'], full_gencode['end'])
full_gencode['gene_end'] = np.where(full_gencode['strand'] == '-', full_gencode['start'], full_gencode['end'])

# filter to just the transcripts that are in the clusters
gene_ids = np.concatenate(overlap_df['cluster_id'].str.split('_'))
gid_gencode = full_gencode.set_index('transcript_id').loc[gene_ids]
gid_gencode = gid_gencode.drop_duplicates()

In [15]:
row = sum_variance_df.iloc[0]

In [19]:
egene_tss = gid_gencode.loc[row['egene_id']]['tss_start']
lead

KeyError: 'egene_id'